In [154]:
import streamlit as st
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pytz
import datetime as datetime
import os
import sys
import numpy as np
import plotly.express as px


In [155]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'FPL')))


In [156]:
from FPL.fpl_api_collection import (
    get_bootstrap_data,
    get_current_gw,
    get_fixt_dfs,
    get_league_table,
    get_fixture_dfs,
    get_fixture_data
)
from FPL.fpl_utils import (
    define_sidebar,
    get_annot_size,
    map_float_to_color,
    get_text_color_from_hash,
    get_rotation
)
from FPL.fpl_params import (
    TIMEZONES_BY_CONTINENT,
    AUTHOR_CONTINENT,
    AUTHOR_CITY
)

In [157]:
team_fdr_df, team_fixt_df, team_ga_df, team_gf_df = get_fixt_dfs()

events_df = pd.DataFrame(get_bootstrap_data()['events'])

gw_min = min(events_df['id'])
gw_max = max(events_df['id'])

ct_gw = get_current_gw()

In [158]:
fixt = team_fixt_df.reset_index()
drf = team_fdr_df.reset_index()
ga = team_ga_df.reset_index()
gf = team_gf_df.reset_index()

# Rename the first column to 'Team'
fixt.rename(columns={0: 'Team'}, inplace=True)
drf.rename(columns={0: 'Team'}, inplace=True)
ga.rename(columns={0: 'Team'}, inplace=True)
gf.rename(columns={0: 'Team'}, inplace=True)

In [159]:
merged_df = pd.merge(fixt, drf, on="Team", suffixes=('_Fixture', '_FDR'))

# Convert to a format suitable for display
merged_display_df = merged_df.copy()
merged_display_df.columns = ["Team"] + [f"{gw}_Fixture" for gw in merged_display_df.columns[1:4]] + [f"{gw}_FDR" for gw in merged_display_df.columns[4:]]

# Streamlit app
st.title("FPL Fixture and Difficulty Rating Analysis")

# Display the merged DataFrame
st.markdown("### Fixture Details with Fixture Difficulty Ratings")
st.dataframe(merged_display_df)

2024-10-17 18:01:12.088 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 18:01:12.089 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 18:01:12.090 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 18:01:12.091 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 18:01:12.099 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 18:01:12.100 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [162]:
saaaa=get_fixture_data()

In [164]:
import json
data = json.loads(saaaa)
fixtures_df = pd.DataFrame(data)

fixtures_df.drop(columns='stats', inplace=True)


AttributeError: 'list' object has no attribute 'text'